# Schedule for one night with respect to objects priority.

## Libraries

In [ ]:
import numpy as np 
import pandas as pd
from astropy.time import Time
from astroplan import Observer
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from pytz import timezone
from tabulate import tabulate
import re
import time
from datetime import date
from astropy.utils.iers import conf
conf.auto_max_age = None #Maximum age (days) of predictive data before auto-downloading. Default is 30.

## Constant part

### Location

In [ ]:
longitude_tshao = '76d55m45.00s'
latitude_tshao = '43d15m49.00s'
elevation_tshao = 2735 * u.m
location_tshao = EarthLocation.from_geodetic(longitude_tshao, latitude_tshao, elevation_tshao)

### Tech time

In [ ]:
# Read-out time
readout_1bin = 20# * u.second
readout_2bin = 14# * u.second
readout_3bin = 10# * u.second

# Filter block turn time
fil_turn_near = 5# * u.second
fil_turn_far = 10# * u.second

# Telescope turning speed
slew_rate = .8 #deg/second 

### Functions

#### Observatory

In [ ]:
class Observatory:
    """
    date                   --- date-time at midnight [Time function, one by one, not list]
                               (if observations are going to be at night, for instance, from 1st to 2d of
                               March 2023 you have to give as input Time("2023-03-02 00:00:00")),
    observatory_name       --- short name of observatory [str],
    longitude, latitude    --- coorinates of observatory [str, "**d**m**.**s"], 
    elevation              --- altitude above sea level [astropy units - meters, by default 0*u.m]
    pressure               --- [astropy units - bar, by default 0*u.bar],
    temperature            --- [astropy units - degrees of C, by default 0*u.deg_C]
    """
    def __init__(self,date, observatory_name, longitude, latitude, elevation=0*u.m,pressure=0*u.bar,temperature=0*u.deg_C):
        
        self.date = date
        self.observatory_name = observatory_name
        self.longitude = longitude
        self.latitude = latitude
        self.elevation_tshao = elevation
        self.pressure = pressure
        self.temperature = temperature
        self.location = EarthLocation.from_geodetic(longitude, latitude, elevation)
        self.observer = Observer(name=self.observatory_name,
                                 location=self.location,
                                 pressure=self.pressure,
                                 temperature=self.temperature)
    
    def sunset_rise_time(self):
    
        """
        ___________
        Description
        ___________
        Calculate sunset and surise time (astronomical twilight). 
        Also, calculate observation start and end in SIDEREAL time 
        
        ______
        Output
        ______
        Returns sunset and rise time as [Time function], sidereal time
        at midnight, sunset and rise (astronomical twilight) moments [float],
        and time range from set to rise time as [list of Time functions].
        All times will be given for UT+0h.
        
        return sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        
        _____
        Input
        _____
        date     --- date-time at midnight [Time function, one by one, not list]
                     (if observations are going to be at night, for instance, from 1st to 2d of
                     March 2023 you have to give as input Time("2023-03-02 00:00:00"))
        observer --- observatory [Astroplan Observer function,one by one, not list] 
        
        >>> sunset_rise_time(Time("2023-03-02 00:00:00"),Observer(location=EarthLocation.from_geodetic('76d58m17.00s', '43d03m29.00s', 2735 * u.m)))
        <Time object: scale='utc' format='jd' value=2460005.093717037>,
        <Time object: scale='utc' format='jd' value=2460005.495337549>,
        <Longitude 5.98912107 hourangle>,
        <Longitude 15.76660942 hourangle>,
        <Longitude 15.65440423 hourangle>,
        [<Time object: scale='utc' format='jd' value=2460005.093717037>,
         <Time object: scale='utc' format='jd' value=2460005.495337549>]
        """
        date = self.date
        observer = self.observer
        
        #Sunset and sunrise local sidereal time (same with strart and end of observation)
        sunset_tonight = observer.sun_set_time(date,  horizon = -18*u.degree, which='nearest') #UT 
        sunrise_tonight = observer.sun_rise_time(date,  horizon = -18*u.degree, which='nearest')  #UT
        
        #Sidereal time at sunset, sunrise and midnight moments
        sid_start =  observer.local_sidereal_time(sunset_tonight).value  #in UT
        sid_end = observer.local_sidereal_time(sunrise_tonight).value #in UT
        sid_mid = observer.local_sidereal_time(date).value #in UT
        
        #time range between sunset and sunrise moments
        time_range = [sunset_tonight, sunrise_tonight]
    
        self.sunset = sunset_tonight
        self.sunrise = sunrise_tonight
        self.sir_start = sid_start
        self.sid_mid = sid_mid
        self.sid_end = sid_end
        self.time_range = time_range
        
        return sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
    
    
    def parking_coordinates(self,date=None):

        """
        ___________
        Description
        ___________
        As we know, telescopes are parked usually on the South and paralell to the graund.
        So we have horizontal coordinates: azimut A = 270 deg and height  h = 0 deg.
        We need only two variable: lattitude of observatory (phi) and date of observations.
        
        ______
        Output
        ______
        Returns equatorial coordinates (RA,DEC) of telescope parking point.
        
        return parking_alpha_deg, parking_delta_deg, parking_alpha_hms, parking_dec_dms
        _____
        Input
        _____
        date     --- date-time at sunset/sunrise moment [Time function, one by one, not list]
        observer --- observatory [Astroplan Observer function,one by one, not list] 
        
        """
        if date:
            date=date
        else:
            date = self.date
            
        observer = self.observer
        
        A = 180*u.degree #because telescope should be parked "looking" to the south
        z = 0*u.degree # because telescope should be parked so that the pipe is parallel to the floor
    
        phi = observer.location.lat
        dec_rad = np.arcsin(np.sin(phi) * np.cos(z) - np.cos(phi) * np.sin(z) * np.cos(A))#* 360 / (2 * 3.14159265)
        parking_dec_deg = dec_rad.to(u.degree)

        JD2 = date.jd #date in JD to future calculations
        T = (JD2 - 2433282.5) / 36524.2    #date in fraction of 100 years
        S0 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T) + (((0.0929 / 60) / 60) * T**2) #sidereal time in Greenwich in UT 
        S0 = S0 % 24 #to account for only 24 hours in a day
        s = S0 + observer.location.lon.value #UT sidereal time in local area 
        s = s%24 #to account for only 24 hours in a day
        sid_h = s - 6 #local sidereal time in local area
        parking_alpha_deg = s * 360 / 24
        
        a_h = int(parking_alpha_deg * 24 / 360)
        a_m = int((parking_alpha_deg * 24 / 360 - a_h) * 60)
        a_s = round(((parking_alpha_deg * 24 / 360 - a_h) * 60 - a_m)*60,2)
        parking_alpha_hms = f'{a_h} {a_m} {a_s}'
    
        d_d = int(parking_dec_deg.value)
        d_m = int((parking_dec_deg.value - d_d) * 60)
        d_s = round(((parking_dec_deg.value - d_d) * 60 - d_m) * 60,2)
        parking_dec_dms = f'{d_d} {d_m} {d_s}'
        
        self.parking_alpha_deg = parking_alpha_deg 
        self.parking_dec_deg = parking_dec_deg 
        self.parking_alpha_hms = parking_alpha_hms 
        self.parking_dec_dms = parking_dec_dms
        
        return parking_alpha_deg, parking_dec_deg, parking_alpha_hms, parking_dec_dms

#### Telescope

In [ ]:
class Telescope:
    
    def __init__(self,readout_1bin=0,readout_2bin=0,readout_3bin=0,fil_turn_near=0,fil_turn_far=0,slew_rate=0):
       
    
        self.readout_1bin = readout_1bin
        self.readout_2bin = readout_2bin
        self.readout_3bin = readout_3bin
        self.fil_turn_near = fil_turn_near
        self.fil_turn_far = fil_turn_far
        self.slew_rate = slew_rate

#### Object

In [ ]:
class Object():
    """
    date          --- date-time at midnight [Time function, one by one, not list]
                      (if observations are going to be at night, for instance, 
                      from 1st to 2d of March 2023 you have to give as input 
                      Time("2023-03-02 00:00:00")),
    ra            --- right ascension of the first object 
                      [float or astropy units in DEGREES, one by one, not list],
    dec           --- declination of the first object 
                      [float or astropy units in DEGREES, one by one, not list],
    B,V,R         --- exposure time in respect filter 
                      [float or int, seconds, one by one, not list],
    bining        --- neccesary binning for object observations 
                      [int, one by one, not list],
    exp_num       --- necessary number of expositions in each filter
                          [int, one by one, not list]
    Telescope     --- Telescope object with some parameters
    Observatory   --- Observatory object with some parameters
    """
    
    def __init__(self,ra,dec, B,V,R,bining,exp_num,Telescope,Observatory):
        self.date = Observatory.date
        self.ra = ra
        self.dec = dec
        self.B = B
        self.V = V
        self.R = R
        self.bining = bining
        self.exp_num = exp_num
        self.Telescope = Telescope
        self.Observatory = Observatory
    
    def dec_to_deg(self,dec=None):
        """
        ___________
        Description
        ___________
        Returns declanation as float in degrees.
        
        ______
        Output
        ______
        deg_deg [float, with respect to sign]
        
        ______
        Input
        ______
        dec -- declenation of object [str, "+-dd:mm:ss, +-dd mm ss"]
        """
          
        DEC_FORMATS = [re.compile(pat) for pat in [
          r"(?P<sign>-?)(?P<degrees>\d+\.?\d*)$",
          r"(?P<sign>-?)(?P<degrees>\d+) (?P<minutes>\d+)(?: (?P<seconds>\d+(?:.\d+)?))?$",
          r"(?P<sign>-?)(?P<degrees>\d+)d(?P<minutes>\d+(?:\.\d+)?)m(?:(?P<seconds>\d+)s?)?$",
          r"(?P<sign>-?)((?P<degrees>\d+):(?P<minutes>\d+):(?P<seconds>\d+(?:.\d+)?))$",
        ]]
        if dec:
            if str(type(dec)) == "<class 'str'>":
                raw_dec = str(dec).strip().replace("+","").replace("'","")
            else:
                return dec
        else:
            if str(type(self.dec)) == "<class 'str'>":
                raw_dec = self.dec.strip().replace("+","").replace("'","")
            else:
                return self.dec
            
        for pattern in DEC_FORMATS:
            mat = re.match(pattern, raw_dec)
            if mat:
                break
        else:
            raise ValueError(f"Not a valid Dec {raw_dec}")
    
        parts = mat.groupdict()
        dec_deg = (float(parts["degrees"])
            + float(parts.get("minutes", 0))/60.
            + float(parts.get("seconds", 0) or 0)/3600.)
    
        if parts["sign"]=="-":
            self.dec_deg = -dec_deg
            return -dec_deg
        else:
            self.dec_deg = dec_deg
            return dec_deg
    
    def ra_to_deg(self,ra=None):
        """
        ___________
        Description
        ___________
        Returns right ascension as float in degrees.
        
        ______
        Output
        ______
        ra_deg [float, degrees]
        
        ______
        Input
        ______
        ra -- right ascension  of object [str, "hh:mm:ss, hh mm ss"]

        """
        RA_FORMATS = [re.compile(pat) for pat in [
          r"(?P<hours>\d+) (?P<minutes>\d+)(?: (?P<seconds>\d+(?:.\d+)?))?$",
          r"(?P<hours>\d+)h(?P<minutes>\d+(?:\.\d+)?)m(?:(?P<seconds>\d+)s)?$",
          r"(?P<hours>\d+)h(?P<minutes>\d+(?:m?\.\d+)?)(?:(?P<seconds>\d+)s)?$",
          r"(?P<hours>\d+):(?P<minutes>\d+):(?P<seconds>\d+(?:.\d+)?)$",
        ]]
        if ra:
            if str(type(ra)) == "<class 'str'>":
                raw_ra = str(ra).strip()
            else:
                return ra
            
        else:
            if str(type(ra)) == "<class 'str'>":
                raw_ra = self.ra.strip()
            else:
                return self.ra
            
        for pattern in RA_FORMATS:
            mat = re.match(pattern, raw_ra)
            if mat:
                break
        else:
            raise ValueError(f"Not a valid RA {raw_ra}")
    
        parts = mat.groupdict()
        hours = (float(parts["hours"])
            + float(parts["minutes"].replace("m",""))/60.
            + float(parts["seconds"] or 0)/3600.)
        self.ra_deg = hours/24*360
        
        return hours/24*360
    
    
    def time_calculation(self):
    
        """
        ___________
        Description
        ___________
        To make schedule we have to know when the object will culminate,
        how much time we should observe it, time of start and finish of
        observations and will be ready to go to the next object or home...
        
        ______
        Output
        ______
        Returns start, end, culmination [Time function] and whole 
        observation time [astropy units - seconds].
        
        return start, end, culm, time
        
        _____
        Input
        _____
        date          --- date-time at midnight [Time function, one by one, not list]
                          (if observations are going to be at night, for instance, 
                          from 1st to 2d of March 2023 you have to give as input 
                          Time("2023-03-02 00:00:00")),
        ra            --- right ascension of the first object 
                          [float or astropy units in DEGREES, one by one, not list],
        dec           --- declination of the first object 
                          [float or astropy units in DEGREES, one by one, not list],
        B,V,R         --- exposure time in respect filter 
                          [float or int, seconds, one by one, not list],
        bining        --- neccesary binning for object observations 
                          [int, one by one, not list],
        exp_num       --- necessary number of expositions in each filter
                          [int, one by one, not list]
        sid_mid       --- sidereal time at midnight on the date of obeservations
                          [float],
        fil_turn_near --- turning the filter wheel; switching to the next closest filter
                          [float/int or astropy units - seconds],
        readout_1bin  --- signal readout time in 1st binning
                          [float/int or astropy units - seconds],
        readout_2bin  --- signal readout time in 2d binning
                          [float/int or astropy units - seconds],
        readout_3bin  --- signal readout time in 3d binning
                          [float/int or astropy units - seconds].
        """
        #FIND CULMINATION TIME IN UT
        #RA is culmination SIDEREAL timew
        ra_deg = Object.ra_to_deg(self,self.ra)
        
        ra_h = ra_deg*360/24
        self.Observatory.sunset_rise_time()
        delta = self.Observatory.sid_mid - ra_h
            
        if abs(delta) < 12:
            culm = self.date - delta*u.second - 6*u.hour
        elif abs(delta) > 12:
            delta = (24 - abs(delta))
            culm = self.date - delta*u.second - 6*u.hour
        #print("Object time_calculation",self.date)
        
        #NECESSARY DURATION OF OBSERVATIONS
        if self.bining == 1:
            time = (self.B*u.second + self.V*u.second + (self.Telescope.fil_turn_near * 2)*u.second + self.R*u.second + (self.Telescope.readout_1bin * 3)*u.second) * self.exp_num#+ fil_turn_far
        elif self.bining == 2: 
            time = (self.B*u.second + self.V*u.second + (self.Telescope.fil_turn_near * 2)*u.second + self.R*u.second  + (self.Telescope.readout_2bin * 3)*u.second) * self.exp_num#+ fil_turn_far
        elif self.bining == 3:
            time = (self.B*u.second + self.V*u.second + (self.Telescope.fil_turn_near * 2)*u.second + self.R*u.second  + (self.Telescope.readout_3bin * 3)*u.second) * self.exp_num#+ fil_turn_far
        
        delta_t = time / 2
        
        #start and end time of observations in respect to object culmination time
        start = culm - delta_t
        end = culm + delta_t
        
        self.start_obj_obs = start
        self.end_obj_obs = end
        self.culm_obj_obs = culm
        self.whole_time_obj_obs = time
        
        return start, end, culm, time
    
    
    def calc_obj_rise_set_time(self):
        
        """
        ___________
        Description
        ___________
        Returns object rise, set time in UT and visible time for Observatory.
        
        ______
        Output
        ______
        time_rise, time_set, visible_time --- [Time functions]
        """
        cos_t = -np.tan(self.Observatory.location.lat)*np.tan(Object.dec_to_deg(self,self.dec)*u.degree)
        td_1 = np.arccos(cos_t)*u.rad.to(u.degree)
        td_2 = -np.arccos(cos_t)*u.rad.to(u.degree)
        
        td_1= td_1.value
        td_2= td_2.value
        
        if td_1 < 0 or td_1 > 360:
            td_1 = td_1%360
        if td_2 < 0 or td_2 > 360:
            td_2 = td_2%360
            
        th_1 = td_1*24/360 
        th_2 = td_2*24/360
            
        if th_2 > 12 and th_2 < 24: #rise point
            hour_angle_hour_rise = th_1
            hour_angle_hour_set = th_2
        else: #set point
            hour_angle_hour_rise = th_2
            hour_angle_hour_set = th_1

        sid_time_rise = (hour_angle_hour_rise + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        sid_time_set = (hour_angle_hour_set + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        
        time_rise = Object.convert_sid2ut(self,sid_time_rise)
        time_set = Object.convert_sid2ut(self,sid_time_set)
            
        visible_time = abs((time_set - time_rise).value) #seconds
        
        self.time_rise = time_rise 
        self.time_set = time_set 
        #print("time_rise, time_set, visible_time ",(time_rise+6*u.hour).iso.split(" ")[1], (time_set+6*u.hour).iso.split(" ")[1], round(visible_time*24,2))
        return time_rise, time_set, visible_time 
        

    def calc_obj_rise_set_15deg_time(self):
        
        """
        ___________
        Description
        ___________
        Returns visible time (in seconds)
        in respect to 15 degree altitude, i.e. when altitude 
        of object became more/less than 15deg.
        
        cos(z)=sin(lat)sin(DEC)+cos(lat)cos(DEC)cos(t)
        z = 90-15
        
        ______
        Output
        ______
        visible_time --- [float, seconds]
        
        """
        cos_t = (np.cos((90-15)*u.degree) - np.sin(self.Observatory.location.lat)*np.sin(Object.dec_to_deg(self,self.dec)*u.degree)) / np.cos(self.Observatory.location.lat)*np.cos(Object.dec_to_deg(self,self.dec)*u.degree)

        td_1 = np.arccos(cos_t)*u.rad.to(u.degree)
        td_2 = -np.arccos(cos_t)*u.rad.to(u.degree)
        
        td_1= td_1.value
        td_2= td_2.value
        
        if td_1 < 0 or td_1 > 360:
            td_1 = td_1%360
        if td_2 < 0 or td_2 > 360:
            td_2 = td_2%360
            
        th_1 = td_1*24/360 
        th_2 = td_2*24/360
            
        if th_2 > 12 and th_2 < 24: #rise point
            hour_angle_hour_rise = th_1
            hour_angle_hour_set = th_2
        else: #set point
            hour_angle_hour_rise = th_2
            hour_angle_hour_set = th_1

        sid_time_rise = (hour_angle_hour_rise + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        sid_time_set = (hour_angle_hour_set + Object.ra_to_deg(self,self.ra)*24/360)%24 # alpha + hour angle
        
        time_rise = Object.convert_sid2ut(self,sid_time_rise)
        time_set = Object.convert_sid2ut(self,sid_time_set)
            
        visible_time = abs((time_set - time_rise).value) #seconds
        
        return visible_time 

    def convert_sid2ut(self,sid_time):
        
        """
        ___________
        Description
        ___________
        Returns date-time as Time function.
        It uses sidereal time at neccessary date
        and then iteratively find exact time 
        by calculating sidereal time at each moment and
        compair it with know sidereal time.
        
        ______
        Output
        ______
        date_final --- [Time function] date-time UT
        """  
        
        date = self.Observatory.date - 6*u.hour#time function
        lst_mid = date.sidereal_time("mean",self.Observatory.location.lon).value #float, UT+0h
        longitude = self.Observatory.location.lon
        date_final = None
        done = False
        hours = -12.5
        
        while done == False:
            hours += 1
            if hours > 11.5:
                print("Not found time")
                done = True
            else:
                date_new = date + hours*u.hour
                st = date_new.sidereal_time("mean",longitude).value #GST
                if int(st)-int(sid_time) == 0: #we get an hour
                    for minutes in range(-60,61):
                        date_minutes = date_new + minutes*u.minute
                        st = date_minutes.sidereal_time("mean",longitude).value#GST
                        if abs(st - sid_time) <= 1/60: #we get a minute, i.e. accuracy is about 1 minute
                            date_final = date_minutes #UT
                            done = True
                            break
                        else:
                            done = False
                else:
                    done = False         
        return date_final 
    
    def decide_ascending_star(self):
        """
        ___________
        Description
        ___________
        Check if object is visible in exact location.
        Visibility means altitude in upper culmination point
        more than 0 deg.
        
        _______
        Output
        _______
        "Circumpolar"   --- [str] never goes under horizont
        "Not ascending" --- [str] never visible in the location
        "Ascending"     --- [str] visible, and goes under horizont
        
        """
        
        dec_deg = Object.dec_to_deg(self)
        
        if dec_deg > 90 - self.Observatory.location.lat.value:
            
            return "Circumpolar"
        elif dec_deg < - (90  - self.Observatory.location.lat.value):
            return "Not ascending"
        else:
            return "Ascending"
        
    def get_culm_up_altitude(self):
        """
        ___________
        Description
        ___________
        Check if object is observable in exact location.
        Observability means altitude in upper culmination point
        more than 15 deg, because under 15deg there is too
        large airmass.
        
        _______
        Output
        _______
        "Visible"        --- [str] upper culmination point above 15deg
        "Not ascending"  --- [str] upper culmination point under 15deg
        """

        dec_deg = Object.dec_to_deg(self)
        latitude = self.Observatory.location.lat.value
        
        altitude = 90 - latitude + dec_deg
        
        if altitude > 15: 
            return "Visible"
        else:
            return "Not ascending"
        
    def get_culm_low_altitude(self):
        """
        ___________
        Description
        ___________
        Check if object is observable in exact location.
        Observability means altitude in upper culmination point
        more than 15 deg, because under 15deg there is too
        large airmass.
        
        _______
        Output
        _______
        "Circumpolar"   --- [str] low culmination point above 15deg
        "Ascending"     --- [str] low culmination point under 15deg
        """
        
        dec_deg = Object.dec_to_deg(self)
        latitude = self.Observatory.location.lat.value
        
        altitude = latitude + dec_deg - 90
        
        if altitude < 15: 
            return "Ascending"
        else:
            return "Circumpolar"    

#### Table with ready schedule

In [ ]:
class ScheduleTable():
    
    def __init__(self,table,Observatory,Telescope):
        
        self.table = table
        self.Observatory = Observatory
        self.date = self.Observatory.date
        self.Telescope = Telescope
        self.table["N"]=self.table["exp_num"]*self.table["M"]
        

    def sort_target_visib(self,ra,dec,B,V,R,binning,exp_num):    
    
        """
        ___________
        Description
        ___________
        Check if RA of an object is in interval [sid_start,sid_end]. 
        In other words, could we observe an object culmination at night.
        Moreovere, we can observe object 2 hour before and after culmination, 
        so it is necessary to take it into account.
        
        ______
        Output
        ______    
        Returns object name [str] and altitude [float] if the object is observable
        and None if it is not observable.
        
        return obj_name, altitude
        _____
        Input
        _____    
        obj_name,ra,dec,phi,sid_start,sid_end
        
        obj_name  --- name of object [str, one by one, not list],
        ra        --- right ascension of object [float in degrees, one by one, not list],
        dec       --- declenation of object [float in degrees, one by one, not list],
        phi       --- latitude of observatory [float in degrees, one by one, not list],
        sid_start --- declenation of object [float, one by one, not list],
        sid_end   --- declenation of object [float, one by one, not list].
        """
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time() #sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sid_start = observatory_time[2]
        sid_end = observatory_time[4]
        target = Object(ra,dec,B,V,R,binning,exp_num, self.Telescope, observatory)
        
        altitude = 90*u.degree + observatory.observer.location.lat.value*u.degree - target.dec_to_deg()*u.degree #formulae for obj on the north from zenith
        if altitude > 90*u.degree:
            altitude = 180*u.degree - altitude #formulae for obj on the south from zenith
            
        if altitude > 15*u.degree and altitude < 80*u.degree: 
            
            ra_h = Object.ra_to_deg(self,target.ra) * 24 / 360
            if sid_start < sid_end:
                if (ra_h > (sid_start-2) and ra_h < (sid_end+2)): #we are able to observe target in range culm +- 2h
                    return round(altitude.value,3)
                else:
                    return None
            else:
                if (ra_h < (sid_end + 2) or ra_h > (sid_start - 2)): #we are able to observe target in range culm +- 2h
                    return round(altitude.value,3)
                else:
                    return None
        else:
            return None
                
                
    def calc_slew_time(self,ra_1,ra_2,dec_1,dec_2,slew_rate):
    
        """
        ___________
        Description
        ___________
        To make schedule we have to know how much time telescope will spend time
        to move from one object to another.
        
        ______
        Output
        ______
        Returns telescope slew time [float, in astropy units - seconds].
        
        return slew_time
        _____
        Input
        _____
        ra_1      --- right ascension of the first object 
                      [float or astropy units in degrees, one by one, not list],
        dec_1     --- declenation of the first object 
                      [float or astropy units in degrees, one by one, not list],
        ra_2      --- right ascension of the second object 
                      [float or astropy units in degrees, one by one, not list],
        dec_2     --- declenation of the second object 
                      [float or astropy units in degrees, one by one, not list],
        slew_rate --- rotate rate of telescope [float! 
                      u.deg/u.second, one by one, not list].
        """

        
        dist = ScheduleTable.calc_distance(self,ra_1,ra_2,dec_1,dec_2)
        
        slew_time = (dist / slew_rate) * u.second
        
        return slew_time
    
    def calc_distance(self,ra_1,ra_2,dec_1,dec_2):
        """
        ___________
        Description
        ___________
        Returns distence in degrees (float, nit astropy units) between
        two points/objects.
        
        _____
        Input
        _____
        ra_1      --- right ascension of the first object 
                      [float or astropy units in degrees, one by one, not list],
        dec_1     --- declenation of the first object 
                      [float or astropy units in degrees, one by one, not list],
        ra_2      --- right ascension of the second object 
                      [float or astropy units in degrees, one by one, not list],
        dec_2     --- declenation of the second object 
                      [float or astropy units in degrees, one by one, not list]       
        ______
        Output
        ______
        distance --- [float]
        """
        
        #Check if the input values are not in astropy units.
        
        if type(ra_1) != "<class 'astropy.units.quantity.Quantity'>":
            ra_1 = Object.ra_to_deg(self,ra_1)*u.degree
        if type(dec_1) != "<class 'astropy.units.quantity.Quantity'>":
            dec_1 = Object.dec_to_deg(self,dec_1)*u.degree
        if type(ra_2) != "<class 'astropy.units.quantity.Quantity'>":
            ra_2 =Object.ra_to_deg(self,ra_2)*u.degree
        if type(dec_2) != "<class 'astropy.units.quantity.Quantity'>":
            dec_2 = Object.dec_to_deg(self,dec_2)*u.degree
        
        distance = (np.arccos(np.sin(dec_1) * np.sin(dec_2) + np.cos(dec_1) * np.cos(dec_2) * np.cos(ra_1 - ra_2))).to(u.degree)
        
        return distance.value
    
    def change_priority_visib(self):
        """
        ___________
        Description
        ___________
        Decide if object is observable or not and then in
        respect to result changes priority.
        If object is not observable -- drop it.
        If object circumpolar -- priority is not changed.
        For all ascending objects it calculates observable time
        and the calculates coefitient k
        k = obs_time[i]/obs_time[max]
        For each ascending object priority is devided by k.
        
        ______
        Output
        ______
        table_ascend_sorted --- pandas table with only observable 
                                objects and their new priorities.
        """
        
        table = self.table
        
        table["priority_vis"] = np.nan
        table["visible_time"] = np.nan
        table["visibility"] = np.nan
        
        for ind in range(len(table)):
            target = Object(table["ra"][ind],table["dec"][ind],table["B"][ind],table["V"][ind],table["R"][ind],table["binning"][ind],table["exp_num"][ind], self.Telescope, self.Observatory)
            table["visible_time"][ind] = target.calc_obj_rise_set_15deg_time()
            if target.get_culm_up_altitude() == "Not ascending":
                table["visibility"][ind] = "Not ascending"
            else:
                table["visibility"][ind] = target.get_culm_low_altitude()
            
        table_ascend = table[table["visibility"]!="Not ascending"].reset_index(drop=True)
        
        maximum = table_ascend[table_ascend["visibility"]=="Ascending"]["visible_time"].max()
        
        for ind in range(len(table_ascend)):
            if table_ascend["visibility"][ind] == "Circumpolar":
                table_ascend["priority_vis"]=table_ascend["priority"]
            else:
                table_ascend["priority_vis"] = table_ascend["priority"]/(table_ascend["visible_time"][ind]/maximum)
        
        table_ascend_sorted = table_ascend.sort_values("ra")
        
        return table_ascend_sorted
        

    def calc_time_table(self,table):
        """
        Returns table with start, end, culmination, 
        and observation time columns in UT.
        """
        table["altitude"] = None        
        
        for index in range(len(table)):
            #check observable or not
            obj_alt = ScheduleTable.sort_target_visib(self,table["ra"][index],table["dec"][index],table["B"][index],table["V"][index],table["R"][index],table["binning"][index],table["exp_num"][index]) 
            if obj_alt == None: #if object is not observable we write Nan altitude to drop the row in the next step
                table["altitude"].iloc[index] = np.nan
            else:#if observable we add altitude data
                table["altitude"].iloc[index] = obj_alt
                continue
                
        #create new table with observable objects only    
        table_timing = table[table["altitude"].notna()].reset_index(drop=True)
        table_timing["start"] = None
        table_timing["end"] = None
        table_timing["culmination"] = None
        table_timing["time_obs"] = None
        
        #now we add new columns with object observation start and end time and object culmination time 
        for index in range(len(table_timing)):
            target = Object(table_timing["ra"][index],table_timing["dec"][index],table_timing["B"][index],table_timing["V"][index],table_timing["R"][index],table_timing["binning"][index],table_timing["exp_num"][index],self.Telescope,self.Observatory)
            table_timing["start"][index],table_timing["end"][index],table_timing["culmination"][index],table_timing["time_obs"][index] = target.time_calculation()
          
        table_timing=table_timing.sort_values(by="ra")
        
        return table_timing

    def check_overlap(self,table=None):
        """
        ___________
        Description
        ___________
        Table with objects which WILL BE observed.
        Check if there overlap in time of obj observation.
        Sorting by percent of made observations, then by priority,
        and finally by culmination time or distance between objecs.
        """
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time() #sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sunset_tonight = observatory_time[0]
        sunrise_tonight = observatory_time[1]
        #print(table)
        try:
            if table==None:
                table = self.table
        except ValueError:
            pass
        prior_table = ScheduleTable.calc_time_table(self,table)
        drop_ind = []
        
        for ind in range(1,len(prior_table)):
            """
            everything is ok, observations step by step ↓
            """
            if prior_table['start'][ind] > prior_table['end'][ind-1]:
                continue
                """
                overlap in time ↓
                """
            else:
                """
                TIME SHIFT IF IND > 1
                """
                if ind > 1: #we calculate ind-2 
                    delta = prior_table['start'][ind-1] - prior_table['end'][ind-2]
                    slew_time_i = ScheduleTable.calc_slew_time(self, prior_table['ra'][ind-1],prior_table['ra'][ind],prior_table['dec'][ind-1],prior_table['dec'][ind],self.Telescope.slew_rate)
                    if delta != 0*u.hour and delta < 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['end'][ind-2] + slew_time_i
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1] 
                    elif delta != 0*u.hour and delta > 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['start'][ind-1] - 2*u.hour  + slew_time_i
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    else: #delta = 0h
                        prior_table['start'][ind] = prior_table['start'][ind] + slew_time_i
                        prior_table['end'][ind] = prior_table['end'][ind] +  prior_table['time_obs'][ind]

                    """
                    TIME SHIFT FOR 2D OBJECT (shift 1st object to sunset)
                    """
                else: #overlap for the 2d object ind = 1
                    delta_sunset = prior_table['start'][ind-1] - sunset_tonight
                    if delta_sunset != 0*u.hour and delta_sunset < 2*u.hour:
                        prior_table['start'][ind-1] = sunset_tonight
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    elif delta_sunset != 0*u.hour and delta_sunset > 2*u.hour:
                        prior_table['start'][ind-1] = prior_table['start'][ind-1] - 2*u.hour
                        prior_table['end'][ind-1] = prior_table['start'][ind-1] + prior_table['time_obs'][ind-1]
                    else: #delta = 0h
                        prior_table['start'][ind-1] = prior_table['start'][ind-1]
                        prior_table['end'][ind-1] = prior_table['start'][ind-1]

                    """
                    CHECK OVERLAP AGAIN
                    """        
                if prior_table['start'][ind] > prior_table['end'][ind-1]:
                    continue

                else:
                    overlap = prior_table['end'][ind-1] - prior_table['start'][ind]
                    if overlap < (prior_table['time_obs'][ind]/4): # 25% 
                        slew_time_i = ScheduleTable.calc_slew_time(self, prior_table['ra'][ind-1],prior_table['ra'][ind],prior_table['dec'][ind-1],prior_table['dec'][ind],self.Telescope.slew_rate)
                        prior_table['start'][ind] = prior_table['start'][ind] + overlap  + slew_time_i
                        prior_table['end'][ind] = prior_table['end'][ind] + overlap  + slew_time_i
                    else:
                        m = int(prior_table['exp_num'][ind - 1])
                        while m > 3: # becuse we need as less 3 frames for each filter to make MasterFrame
                            #it relates with statistic issues 
                            m -= 1
                            if prior_table["binning"][ind-1]==1:
                                obs_time_1 = prior_table['B'][ind-1]*u.second + prior_table['V'][ind-1]*u.second + fil_turn_near*u.second * 2 + prior_table['R'][ind-1]*u.second + fil_turn_far*u.second + (self.Telescope.readout_1bin * 3)*u.second
                            elif prior_table["binning"][ind-1]==1:
                                obs_time_1 = prior_table['B'][ind-1]*u.second + prior_table['V'][ind-1]*u.second + fil_turn_near*u.second * 2 + prior_table['R'][ind-1]*u.second + fil_turn_far*u.second + (self.Telescope.readout_2bin * 3)*u.second
                            else:
                                obs_time_1 = prior_table['B'][ind-1]*u.second + prior_table['V'][ind-1]*u.second + fil_turn_near*u.second * 2 + prior_table['R'][ind-1]*u.second + fil_turn_far*u.second + (self.Telescope.readout_3bin * 3)*u.second

                            prior_table['end'][ind-1] = prior_table['end'][ind-1] - obs_time_1
                            overlap_m = prior_table['end'][ind-1] - prior_table['start'][ind]
                            #N - 1 worked
                            if overlap_m < (prior_table['time_obs'][ind]/4):#25%
                                slew_time_i = ScheduleTable.calc_slew_time(self, prior_table['ra'][ind-1],prior_table['ra'][ind],prior_table['dec'][ind-1],prior_table['dec'][ind],self.Telescope.slew_rate)
                                prior_table['exp_num'][ind - 1] = m
                                prior_table['start'][ind] = prior_table['start'][ind] + overlap_m + slew_time_i
                                prior_table['end'][ind] = prior_table['end'][ind] + overlap_m + slew_time_i
                                break
                            #N - 1 didn't work. Repeat
                            else:
                                continue

                        # N < 3, we can't observe less than 3 series
                        # so we should take into accaunt priority
                        else:
                            """
                            Check percent of observation which were made
                            """
                            if float(prior_table['N_obs_perc'][ind-1]) > float(prior_table['N_obs_perc'][ind]):
                                drop_ind.append(ind)
                            elif float(prior_table['N_obs_perc'][ind-1]) < float(prior_table['N_obs_perc'][ind]):
                                drop_ind.append(ind-1)
                            else:
                                """
                                If percent of observation which were made is equal.
                                Check priority.
                                First off all, we want to give some observations for 
                                all type of objects (for all orders),
                                so we try to do observations homogeniously
                                But if the percent is equal - we have to look at priority.
                                """
                                if float(prior_table['priority'][ind-1]) > float(prior_table['priority'][ind]):
                                    drop_ind.append(ind)
                                elif float(prior_table['priority'][ind-1]) < float(prior_table['priority'][ind]):
                                    drop_ind.append(ind-1)
                                else:
                                    """
                                    If priority is equal and if we can calculate distances among 3 stars, we will choose 
                                    the nearest star, but if we cannot calculate distances we will choose 
                                    that star which culminate first.
                                    """
                                    if ind < 2:
                                        #if we cannot calculate distances among 3 stars, we will choose 
                                        #that star which culminate first
                                        if prior_table['start'][ind-1] > prior_table['start'][ind]: 
                                            drop_ind.append(ind)
                                        elif prior_table['start'][ind-1] < prior_table['start'][ind]:
                                            drop_ind.append(ind-1)
                                    else: # ind >= 2
                                        #if we can calculate distances among 3 stars, we will choose 
                                        #the nearest star
                                        distance_20 = ScheduleTable.calc_distance(self,prior_table['ra'][ind-2],prior_table['ra'][ind],prior_table['dec'][ind-2],prior_table['dec'][ind])
                                        distance_21 = ScheduleTable.calc_distance(self,prior_table['ra'][ind-2],prior_table['ra'][ind-1],prior_table['dec'][ind-2],prior_table['dec'][ind-1])
                                        if distance_20 > distance_21:
                                            drop_ind.append(ind)
                                        else:
                                            drop_ind.append(ind-1)
                
                """
                DROP objects, which will not observe this night
                """
        prior_table = prior_table.drop(labels = drop_ind).reset_index(drop=True)
    
        """
        Check if observations is duiring after sunrise
        """
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time()#sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sunrise_tonight = observatory_time[1]
        if prior_table['end'].iloc[len(prior_table)-1] > sunrise_tonight:
            slew_end_parking_overlap = ScheduleTable.calc_slew_time(self,prior_table['ra'].iloc[len(prior_table)-2], prior_table['ra'].iloc[len(prior_table)-2],prior_table['dec'].iloc[len(prior_table)-1], prior_table['dec'].iloc[len(prior_table)-2],self.Telescope.slew_rate) 
            prior_table['end'].iloc[len(prior_table)-1] = prior_table['end'].iloc[len(prior_table)-2] + slew_end_parking_overlap
            prior_table = prior_table.drop(labels=len(prior_table)-1, axis=0)
            if prior_table['end'].iloc[len(prior_table)-1] > sunrise_tonight:
                print("Observations are shifted too much")
            
            """
            Make new table with time without date.
            """
        prior_table_splitted = prior_table
        
        prior_table_splitted = prior_table_splitted.reset_index(drop=True)
        
        return prior_table_splitted
    
    def two_tables(self):
        """
        Takes table with data on objects.
        Returns two tables:
        
        full_table      -- table with objects with new priorities 
        prior_table_spl -- schedule for night
        """
        
        observatory = self.Observatory
        observatory_time = observatory.sunset_rise_time()#sunset_tonight, sunrise_tonight, sid_start, sid_mid, sid_end, time_range
        sunrise_tonight = observatory_time[1]
        sunset_tonight = observatory_time[0]
        
        full_table =  ScheduleTable.change_priority_visib(self) # drop not observable objects and sort by ra
        prior_table_spl = ScheduleTable.check_overlap(self,table=full_table)
        
        prior_table_spl['N_obs_perc'] = round(prior_table_spl['exp_num']/prior_table_spl['N'],2)
        prior_table_spl['priority'] = round(prior_table_spl['priority'] - prior_table_spl['N_obs_perc'],2)
        
        for j in range(0,len(prior_table_spl)):
            for i in range(0,len(full_table)):
                if full_table['name'].iloc[i] == prior_table_spl['name'].iloc[j]:
                    full_table['priority'].iloc[i] = prior_table_spl['priority'].iloc[j]
                    full_table['N_obs_perc'].iloc[i] = prior_table_spl['N_obs_perc'].iloc[j]
                    
        #Find equatorial coordinates (RA,DEC) of telescope parking point.
        parking_alpha_deg_end, parking_dec_deg_end, parking_alpha_hms_end, parking_dec_dms_end = self.Observatory.parking_coordinates(date=sunrise_tonight)
        
        slew_parking_end=ScheduleTable.calc_slew_time(self,prior_table_spl["ra"].iloc[-1],parking_alpha_hms_end,prior_table_spl["dec"].iloc[-1],parking_dec_dms_end,self.Telescope.slew_rate)
        
        #drop unnecessary columns
        prior_table_spl = prior_table_spl.drop(["priority_vis","visible_time","visibility","priority","M","N_obs_perc","N"],axis=1)
        full_table = full_table.drop(["priority_vis","visible_time","visibility","altitude","N"],axis=1).reset_index(drop=True)
        
        # Parking point row. We should add it take into account slew time at the beginning and end of observation 
        obs_end = pd.DataFrame({'name':'parking_point', 'ra':parking_alpha_hms_end, 'dec':parking_dec_dms_end, 'B':np.NaN, 'V':np.NaN, 'R':np.NaN, 'binning':np.NaN, 'exp_num':np.NaN, 'start':np.NaN, 'end':prior_table_spl["end"].iloc[-1]+slew_parking_end, 'culmination':np.NaN, 'time_obs':np.NaN}, index =[0])
        
        prior_table_spl = pd.concat([prior_table_spl,obs_end]).reset_index(drop = True)
        
        for ind in range(0,len(prior_table_spl)-1):#
            prior_table_spl["start"][ind] =  prior_table_spl["start"][ind].iso.split(" ")[1]
            prior_table_spl["end"][ind] = prior_table_spl["end"][ind].iso.split(" ")[1]
            prior_table_spl["culmination"][ind] = prior_table_spl["culmination"][ind].iso.split(" ")[1]
            
        prior_table_spl["end"].iloc[-1] = prior_table_spl["end"].iloc[-1].iso.split(" ")[1]  

        return full_table, prior_table_spl

## Variable part

### Date

In [ ]:
obs_date = Time('2023-03-01 00:00:00') #date of observations + 1 (because we need midnight)
#if we well make observations in night between 14th and 15th of February, we should write '2022-02-15 00:00:00'

In [ ]:
regulus= Object("10 09 23.21","11 53 12", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

atria = Object("16 48 39.21","-69 53 12", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

kohlab = Object("16 48 39.21","74 09 00", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

aldhanab= Object("10 09 23.21","-37 21 57", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 17:44  --- 01.03 07:13

sirius= Object("06 46 10.25","-16 45 03.2", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 16:12  ---  01.03 01:58

procyon= Object("07 39 17","5 12 59", 90,60,30,1,5,TELESCOPE,TSHAO) # 28.02 15:41  ---  01.03 04:18

### Import data

In [ ]:
data = pd.read_excel('objects_example_test.xlsx')

### Calculations

#### Objects

In [ ]:
TELESCOPE =Telescope(readout_1bin,readout_2bin,readout_3bin,fil_turn_near,fil_turn_far,slew_rate)

TSHAO = Observatory(obs_date,"tshao",longitude_tshao,latitude_tshao)

table = ScheduleTable(data,TSHAO,TELESCOPE)

#### Tables

In [ ]:
full_table, prior_table_spl = table.two_tables()

In [ ]:
full_table

In [ ]:
prior_table_spl

## Save tables

In [ ]:
#Save table with plan at night
d = obs_date.value.split(" ")[0]

prior_table_spl.to_excel(f'{d}_schedule.xlsx',index=False)
#Save table with all objects

full_table.to_excel(f"{date.today().strftime('%Y-%m-%d')}_objs_new_prior.xlsx",index=False)

## tabulating in LaTeX format

In [ ]:
print(tabulate(prior_table_spl, headers=prior_table_spl.columns, tablefmt="latex"))